In [1]:
import pandas as pd
import json
import unicodedata
import transformers

In [2]:
df = pd.read_csv("cleanedData1.csv")

In [3]:
len(df)

37222

In [4]:
def explore_units(df):
    ingredient_unit_count_map = dict()
    for index, row in df.iterrows():
        ingredients = json.loads(row["ingredients"])
        for ingredient in ingredients:
            if "unit" in ingredient:
                unit = ingredient["unit"]
                if unit == "":
                    unit = "NONE"
                if unit in ingredient_unit_count_map:
                    ingredient_unit_count_map[unit] += 1
                else:
                    ingredient_unit_count_map[unit] = 1

    return ingredient_unit_count_map

In [58]:
def get_ingredient_name_mapping(df):
    ingredient_count_map = dict()
    for index, row in df.iterrows():
        ingredients = json.loads(row["ingredients"])
        for ingredient in ingredients:
            if "ingredient" in ingredient:
                ing_name = ingredient["short_name"]
                if ing_name == "":
                    ing_name = "NONE"
                if ing_name not in ingredient_count_map:
                    ingredient_count_map[ing_name] = ""
    return ingredient_count_map

In [158]:
def convert_to_decimal(num_str):
    try:
        if "pinch of" in num_str:
            return num_str
        if "1 to 2" in num_str or "1-2" in num_str:
            return 1.5
        
        if num_str == "1 1⁄8":
            return 1.125

        final_num = 0
        mult = 1
        
        try:
            return float(num_str)
        except:
            pass

        if "/" in num_str:
            split_on_space = num_str.split(" ")
            res = 0
            for num_part in split_on_space:
                if num_part != "":
                    if "/" in num_part:
                        res += int(num_part.split("/")[0]) / int(num_part.split("/")[1])
                    else:
                        res += int(num_part)
            return res
        elif "⁄" in num_str:
            split_on_space = num_str.split(" ")
            res = 0
            for num_part in split_on_space:
                if num_part != "":
                    if "⁄" in num_part:
                        res += int(num_part.split("⁄")[0]) / int(num_part.split("⁄")[1])
                    else:
                        res += int(num_part)
            return res
        else:
            for i in reversed(range(len(num_str))):
                if num_str[i] != " ":
                    num = unicodedata.numeric(num_str[i])
                    final_num += (num*mult)
                    if num >= 1 or num == 0:
                        mult *= 10
                        
            return final_num
    except:
        print(num_str)
        raise("error")
        
            
    

In [182]:
FOODS_IN_QUANTITY = ["green bell peppers, cut into large chunks", "jalapeño", "large"]
IGNORE_QUANTITY = ["several", "chunks"]
UNIT_IN_QUANTITY = ["tablespoons", "oz"]

new_ingredients = []

for index, row in df.iterrows():
    ingredients = json.loads(row["ingredients"])
    for ingredient in ingredients:
        if "quantity" in ingredient:
            qt = ingredient["quantity"]
            if type(qt) is str:
                qtn = 0.0
                if " hard-cooked" in qt:
                    qt = qt.replace(" hard-cooked", "")
                if "cups" in qt:
                    qt = qt.replace("cups", "")
                    ingredient["unit"] = "cups"
                if "cup" in qt:
                    qt = qt.replace("cup", "")
                    ingredient["unit"] = "cup"

                if qt == "13 ¾ to 15":
                    qt = "15"

                if "to" in qt:
                    qt = qt.split("to")[1]
                    qt = qt.replace(" ", "")

                if "2 R" in qt:
                    ingredient["ingredient"] = "R" + ingredient["ingredient"]
                    qt = "2"

                for uic in UNIT_IN_QUANTITY:
                    if uic in qt:
                        ingredient["unit"] = uic
                        qt = qt.replace(uic, "")
                        qt = qt.replace(" ", "")

                for fic in FOODS_IN_QUANTITY:
                    if fic in qt:
                        qt = qt.replace(fic, "")

                if qt == "1 (16":
                    qt = "1"
                    ingredient["unit"] = "(16 " + ingredient["unit"]

                if qt == "1 (5 ½ pound)":
                    qt = "5.5"
                    ingredient["unit"] = "pound"

                if qt in IGNORE_QUANTITY:
                    qtn = 0
                else:
                    qt = qt.replace("\xa0", " ")
                    qtn = convert_to_decimal(qt)
                ingredient["quantity"] = qtn
    new_ingredients.append(json.dumps(ingredients))
df["ingredients"] = new_ingredients

In [203]:
def get_package_to_amt_dict(df, unit_str):
    ounce_package_dict = dict()
    units = explore_units(df).keys()
    for unit in units:
        if unit_str in unit:
            try:
                oz_num = unit

                oz_num = oz_num.replace("(", "")
                oz_num = oz_num.replace(")", "")
                oz_num = oz_num.replace("-", " ")

                oz_num = oz_num.split(" " + unit_str)[0]
                oz_num = convert_to_decimal(oz_num)
                ounce_package_dict[unit] = oz_num
            except:
                pass

    return ounce_package_dict

In [208]:
pound_package_dict = get_package_to_amt_dict(df, "pound")
print(pound_package_dict)

pounds
pound
2 to 3
medium 1 ½
4 to 6
3 to 4
2 1/2 to 3
3 to 3 1/2
small 2
small 1
pound boneless,
4pounds
large 2
pound unsalted
6 to 7
10 to 12
3 to 3½
pound 2 cups
2½ to 3
½pound
{'(2 pound)': 2.0, '(3 pound)': 3.0, '(1 pound) loaves': 1.0, '(3 1/2) pound': 3.5, '(1 pound)': 1.0, '(1 1/2-pound)': 1.5, '(4 3/4-pound)': 4.75, '(4 pound)': 4.0, '(1/2 pound)': 0.5, '(5 pound)': 5.0, '(2 ½ pound)': 2.5, '(8 pound)': 8.0, '(6 pound)': 6.0, '(2 1/2 pound)': 2.5, '(10 pound)': 10.0, '(1 pound) loaf': 1.0, '(12 pound)': 12.0, '(2.5 pound)': 2.5, '(3/4 pound)': 0.75, '(2 pound) package': 2.0, '(15 pound)': 15.0, '(3 pound) package': 3.0, '(1 1/4 pound)': 1.25, '(12-pound)': 12.0, '(1 pound) package': 1.0, '(1.5 pound) head': 1.5, '(1 1/2) pound': 1.5, '(1 1/2) pounds': 1.5, '(4.5 pound)': 4.5, '(3 1/2 pound)': 3.5, '(4 pound) package': 4.0, '(3 pound) bag': 3.0, '(1 1/2 pound)': 1.5, '(16 pound)': 16.0, '(1 pound) packages': 1.0, '(7 pound)': 7.0, '(14 pound)': 14.0, '(1 ½-pound)': 1.5, '(2 p

In [62]:
df.to_csv("cleanedData2.csv", index=False)


In [209]:
new_ingredients = []
for index, row in df.iterrows():
        ingredients = json.loads(row["ingredients"])
        for ingredient in ingredients:
            if "unit" in ingredient:
                unit = ingredient["unit"]
                if unit in pound_package_dict:
                    if "quantity" not in ingredient:
                        ingredient["quantity"] = 1.0
                    if type(ingredient["quantity"]) == str:
                        ingredient["quantity"] = float(ingredient["quantity"])
                    qty = ingredient["quantity"] * pound_package_dict[unit]
                    ingredient["quantity"] = qty
                    ingredient["unit"] = "pound"
                    ingredient["suggested unit"] = unit
        new_ingredients.append(json.dumps(ingredients))
df["ingredients"] = new_ingredients

In [214]:
explore_units(df)

{'cup': 80414,
 'pounds': 4143,
 'ounce': 26002,
 'large': 8131,
 'cups': 29357,
 'teaspoon': 54292,
 'teaspoons': 14467,
 'tablespoons': 30828,
 'NONE': 41158,
 'tablespoon': 21326,
 'pound': 7792,
 'dash': 542,
 'small': 2102,
 'medium': 3393,
 'sprigs': 491,
 'clove': 1563,
 'cloves': 5115,
 'cubes': 272,
 'ounces': 4199,
 'quart': 496,
 '(9 inch)': 521,
 'heads': 118,
 'large cloves': 118,
 'head': 641,
 'ribs': 62,
 'slices': 1988,
 'cube': 170,
 'quarts': 414,
 'stalks': 835,
 'gallon': 144,
 'large head': 128,
 'bunch': 1019,
 'small head': 121,
 'pinch': 4524,
 'medium head': 149,
 'stalk': 310,
 'large clove': 73,
 'ears': 149,
 '(2 inch) piece': 59,
 'ear': 16,
 'pinches': 154,
 'pint': 365,
 'large stalks': 25,
 '(2 to 3 pound)': 39,
 'packet': 42,
 '(750 milliliter) bottle': 83,
 'leaves': 243,
 'thick slice': 3,
 'small bunch': 34,
 'bunches': 131,
 'small (4 inch)': 6,
 'gallons': 29,
 'fluid ounces': 574,
 'small heads': 9,
 '(8 inch)': 101,
 'strips': 141,
 'dashes': 15

In [61]:
ing_names_dict = get_ingredient_name_mapping(df)
print(len(ing_names_dict.keys()))
print(ing_names_dict)

13033
{'onion': '', 'ground beef': '', 'sauerkraut': '', 'refrigerated crescent rolls': '', 'Cheddar cheese': '', 'eggs': '', 'vegetable oil': '', 'white sugar': '', 'all-purpose flour': '', 'baking soda': '', 'cinnamon': '', 'salt': '', 'vanilla extract': '', 'apples - peeled': '', 'walnuts': '', 'cream cheese': '', 'butter': '', "confectioners' sugar": '', 'egg whites': '', 'coconut ': '', 'olive oil for brushing': '', 'olive oil': '', 'honey': '', 'kosher salt': '', 'ground black pepper': '', 'tri-colored baby carrots': '', 'balsamic glaze': '', 'chopped fresh chives': '', 'sweet potatoes': '', 'unsalted butter': '', 'cayenne pepper': '', 'ginger': '', 'maple syrup': '', 'Brussels sprouts': '', 'mayonnaise': '', 'mustard': '', 'eggplant': '', 'zucchini': '', 'tomato': '', 'bell pepper,': '', 'cayenne pepper,': '', 'basil,': '', 'oregano,': '', 'garlic': '', 'black pepper ': '', 'wine': '', 'vinegar': '', 'cooking spray': '', 'pork ': '', 'teriyaki sauce': '', 'rice vinegar': '', 'cr

In [35]:
from transformers import pipeline

pipe = pipeline("token-classification", model="yanekyuk/bert-keyword-extractor")

config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use mps:0


In [45]:
emb = pipe('stemmed and chopped kale')

In [46]:
emb

[{'entity': 'B-KEY',
  'score': np.float32(0.9891332),
  'index': 6,
  'word': 'ka',
  'start': 20,
  'end': 22},
 {'entity': 'I-KEY',
  'score': np.float32(0.91615254),
  'index': 7,
  'word': '##le',
  'start': 22,
  'end': 24}]

In [47]:
for key in ing_names_dict.keys():
    seq = pipe(key)
    if len(seq) > 0:
        start = seq[0]["start"]
        end = seq[-1]["end"] 
        #print(key[start-1:end+1])
        ing_names_dict[key] = key[start-1:end+1]

KeyboardInterrupt: 

In [49]:
def map_keys(ing_names_dict, keys):
    emb = pipe(keys)
    for i, seq in enumerate(emb):
        if len(seq) > 0:
            start = seq[0]["start"]
            end = seq[-1]["end"] 
            ing_names_dict[keys[i]] = keys[i][start-1:end+1]

keys_to_process = []
BATCH_SIZE = 50
i = 0
for key in ing_names_dict.keys():
    keys_to_process.append(key)
    if len(keys_to_process) == BATCH_SIZE:
        map_keys(ing_names_dict, keys_to_process)
        keys_to_process = []
        print(i)
        i += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [51]:
for key in ing_names_dict.keys():
    if len(ing_names_dict[key]) <= 2:
        ing_names_dict[key] = key.split(",")[0]
    else:
        ing_names_dict[key] = ing_names_dict[key].lstrip()

In [52]:
print(ing_names_dict)

{'chopped onion': 'onion', 'lean ground beef': 'ground beef', 'sauerkraut, drained and pressed dry': 'sauerkraut', 'refrigerated crescent rolls': 'refrigerated crescent rolls', 'shredded Cheddar cheese': 'Cheddar cheese', 'eggs': 'eggs', 'vegetable oil': 'vegetable oil', 'white sugar': 'white sugar', 'all-purpose flour': 'all-purpose flour', 'baking soda': 'baking soda', 'ground cinnamon': 'cinnamon', 'salt': 'salt', 'vanilla extract': 'vanilla extract', 'apples - peeled, cored and diced': 'apples - peeled', 'chopped walnuts': 'walnuts', 'cream cheese': 'cream cheese', 'butter, softened': 'butter', "confectioners' sugar": "confectioners' sugar", 'egg whites': 'egg whites', 'unsweetened coconut flakes': 'coconut ', 'olive oil for brushing': 'olive oil for brushing', 'olive oil': 'olive oil', 'honey': 'honey', 'kosher salt': 'kosher salt', 'ground black pepper': 'ground black pepper', 'tri-colored baby carrots': 'tri-colored baby carrots', 'balsamic glaze': 'balsamic glaze', 'butter': 'b

In [57]:
new_ingredients = []
for index, row in df.iterrows():
        ingredients = json.loads(row["ingredients"])
        for ingredient in ingredients:
            ing_name = ingredient["ingredient"]
            short_name = ing_names_dict[ing_name] if ing_name in ing_names_dict else ""
            ingredient["short_name"] = short_name
        new_ingredients.append(json.dumps(ingredients))
df["ingredients"] = new_ingredients
                

In [60]:
new_ingredients = []
for index, row in df.iterrows():
        ingredients = json.loads(row["ingredients"])
        for ingredient in ingredients:
            short_name = ingredient["short_name"]
            ingredient["short_name"] = short_name.split(" (")[0]
        new_ingredients.append(json.dumps(ingredients))
df["ingredients"] = new_ingredients